<a href="https://colab.research.google.com/github/shreyash53/SMAI-Knowledge-Distilation/blob/main/Knowledge_Distilation_in_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow import keras

In [5]:
# Importing MNIST dataset

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [6]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("Train data shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

Train data shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [7]:
from sklearn.preprocessing import OneHotEncoder

print ('label shape: ',np.shape(y_train),np.shape(y_test))
def encode(y):
    df = pd.DataFrame(y, columns=["class"])
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoder_df = pd.DataFrame(encoder.fit_transform(df[['class']]).toarray())
    final_df = df.join(encoder_df)
    final_df.drop('class', axis=1, inplace=True)
    return final_df.to_numpy()
y_train = encode(y_train)
y_test = encode(y_test)
print ('label shape: ',np.shape(y_train),np.shape(y_test))
# print(y_test_[2], y_test[2])

label shape:  (60000,) (10000,)
label shape:  (60000, 10) (10000, 10)


In [4]:
# Teacher Model

teacher_model = keras.Sequential([
    keras.Input(shape=(28, 28, 1)),
    keras.layers.Conv2D(256, kernel_size=(3, 3), padding="same", activation="relu"),
    keras.layers.Conv2D(512, kernel_size=(3, 3), padding="same", activation="relu"),
    keras.layers.Flatten(),
    keras.layers.Dense(10),
    ], name="teacher"
)

teacher_model.summary()

Model: "teacher"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 256)       2560      
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 512)       1180160   
                                                                 
 flatten (Flatten)           (None, 401408)            0         
                                                                 
 dense (Dense)               (None, 10)                4014090   
                                                                 
Total params: 5,196,810
Trainable params: 5,196,810
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
teacher_model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
teacher_model.fit(x_train, y_train, epochs=20, validation_split=0.1, callbacks=[callback])